In [417]:
import pandas as pd
import numpy as np
import requests
import os
from google.cloud import bigquery
from google.oauth2 import service_account

In [418]:
##credenciais para enviar para bigquery
credentials=service_account.Credentials.from_service_account_file(filename='GBQ.json',
                                                                 scopes=["https://www.googleapis.com/auth/cloud-platform"])


In [419]:
## conectar na api de filmes
url = "https://api.themoviedb.org/3/movie/upcoming?api_key="

headers = {
    "accept": "application/json",
    "Authorization": "Bearer chave"
}

response = requests.get(url, headers=headers)
dados=response.json()
dt = pd.json_normalize(dados,'results') 

In [420]:

listafilmes=pd.DataFrame(dt)
listafilmes = listafilmes.astype({'original_language':'string','overview':'string'})  
listafilmes.drop(columns=['adult','backdrop_path','genre_ids','poster_path','video'],inplace=True) #tira coluna
listafilmes.rename(columns={'original_language':'Origem'}|
                           {'original_title':'Titulo_Original'}|
                           {'overview':'Descricao'}|
                           {'popularity':'Popularidade'}|
                           {'release_date':'data_estreia'}|
                           {'title':'Titulo'}|
                           {'vote_average':'Media'}|
                           {'vote_count':'Votos'}
                                ,inplace=True)
listafilmes 


,id,Origem,Titulo_Original,Descricao,Popularidade,data_estreia,Titulo,Media,Votos
0,507089,en,Five Nights at Freddy's,"Recently fired and desperate for work, a troub...",5818.278,2023-10-25,Five Nights at Freddy's,8.400,1215
1,678512,en,Sound of Freedom,"The story of Tim Ballard, a former US governme...",861.649,2023-07-03,Sound of Freedom,8.138,1069
2,901362,en,Trolls Band Together,"When Branch’s brother, Floyd, is kidnapped for...",312.559,2023-10-12,Trolls Band Together,6.700,20
3,744857,es,Cuando acecha la maldad,Residents of a small rural town discover that ...,235.767,2023-10-05,When Evil Lurks,7.728,92
4,663712,en,Terrifier 2,"After being resurrected by a sinister entity, ...",199.499,2022-10-06,Terrifier 2,6.800,1457
5,609681,en,The Marvels,"Carol Danvers, aka Captain Marvel, has reclaim...",198.640,2023-11-08,The Marvels,0.000,0
6,912908,en,Strays,When Reggie is abandoned on the mean city stre...,189.713,2023-08-17,Strays,7.500,470
7,944952,en,The Royal Hotel,After running out of money while backpacking i...,160.287,2023-10-06,The Royal Hotel,5.879,28
8,508883,ja,君たちはどう生きるか,"While the Second World War rages, the teenage ...",106.745,2023-07-14,The Boy and the Heron,7.635,52
9,1145851,es,El hombre del saco,Explores the Spanish origins of the character'...,96.509,2023-08-11,El hombre del saco,5.594,32


In [421]:
listafilmes.reset_index(drop=True,inplace=True)
listafilmes.tail()



,id,Origem,Titulo_Original,Descricao,Popularidade,data_estreia,Titulo,Media,Votos
15,753342,en,Napoleon,A personal look at the French military leader’...,78.034,2023-11-22,Napoleon,0.0,0
16,1022796,en,Wish,"Asha, a sharp-witted idealist, makes a wish so...",74.999,2023-11-22,Wish,0.0,0
17,792293,en,Dumb Money,Vlogger Keith Gill sinks his life savings into...,57.890,2023-09-15,Dumb Money,6.8,41
18,956920,no,There's Something in the Barn,An American family fulfills their dream of mov...,56.421,2023-11-02,There's Something in the Barn,0.0,0
19,943822,en,Prizefighter: The Life of Jem Belcher,"At the turn of the 19th century, Pugilism was ...",55.549,2022-06-30,Prizefighter: The Life of Jem Belcher,6.4,156


In [422]:
#listafilmes.dtypes
# criar tabela do zero
project_id = "meus-projetos-403821" ## projeto
table_id = 'Dashbord.filmes_api' # tabela de destino no dataset
listafilmes.to_gbq(table_id, 
                        project_id=project_id,
                        if_exists='replace',
                         table_schema=[{'name':'data_entreia','type':'DATE'}
                                        ],
                        credentials=credentials)
